<a href="https://colab.research.google.com/github/Keoni808/KaggleCourses/blob/main/Intermediate_Machine_Learning/Missing_values/Missing_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount your Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -rT '/content/drive/MyDrive/Colab Notebooks/KaggleCourses/Intermediate Machine Learning/Missing Values/MV_DATA' .
# !cp '/content/drive/MyDrive/Colab Notebooks/KaggleCourses/Geospatial Analysis/Coordinate Reference Systems/purple_martin.csv' .

In [3]:
# imports

# Data maniupulation
import pandas as pd

# Models
from sklearn.ensemble import RandomForestRegressor

# Model help
from sklearn.metrics import mean_absolute_error #for scoring
from sklearn.model_selection import train_test_split #for spliting training/testing data
from sklearn.impute import SimpleImputer

#Three Approaches


## 1 - Drop Columns with Missing Values
A Simple Option
- drops column with a missing entry

## 2 - Imputation
A Better Option
- fills in missing values with some number.
  - For example: putting in the mean value in for mising values
    - Won't be correct but accuracy will go up

## 3 - An Extension to Imputation
- Imputation but noting which values were originally missing.
    - Potentially create a new column with boolean values


# Example
- Model will focus on number of rooms and land size to predict home price

In [4]:
# loading data
data = pd.read_csv('melb_data.csv')

# Select target
y = data.Price

# To keep thing simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# print(len(data.columns))
# print(len(melb_predictors.columns))
# print(len(X.columns))

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size = 0.2, random_state = 0)

## Define Function

In [5]:
# Define Function to Measure Quality of Each Approach
# score_dataset()
#   Will give MAE from a random forest model

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
  model = RandomForestRegressor(n_estimators=10, random_state=0)
  model.fit(X_train, y_train)
  preds = model.predict(X_valid)
  return mean_absolute_error(y_valid, preds)

## Approach 1
- Drop columns with missing values

In [6]:
# CAUTION:
#   Since we are working with both training and validation sets,
#   we need to be sure that we drop the same tables for both of them

# Get names of columns with missing values
cols_with_missing = [
    col for col in X_train.columns
    if X_train[col].isnull().any()
    ]

# print(cols_with_missing)

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


##Approach 2
- Imputation (replace missing with avg)

In [7]:
# SimpleImputer
# Tool used for imputation

# There are other form of imputation such as "regression imputation"
# They generally have no effect on machine learning models

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178166.46269899711


## Approach 3
- An extension to imputation

In [8]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
  X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
  X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus)) #why fit_transform?
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus)) #why transform?

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


In [9]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


#EXERCISE EXAMPLE QUESTIONS

In [18]:
#SETUP

# Read the data
X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True) # dropped all null values in SalePrice
y = X_full.SalePrice # Grabbed SalePrice column
X_full.drop(['SalePrice'], axis=1, inplace=True) #Dropped SalePrice for training

# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object']) #Dropped all columns with type object
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [19]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


In [20]:
# Step 1: Preliminary investigation

# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column.sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
276
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [21]:
# Part A

# Fill in the line below: How many rows are in the training data?
num_rows = 1168

# Fill in the line below: How many columns in the training data
# have missing values?
num_cols_with_missing = 3

# Fill in the line below: How many missing entries are contained in
# all of the training data?
tot_missing = 276

In [22]:
# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [23]:
# Step 2: Drop columns with missing values

# Fill in the line below: get names of columns with missing values
cols_with_missing = [
    col for col in X_train.columns
    if X_train[col].isnull().any()
    ] # Your code here

print(cols_with_missing)

# # Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis="columns")
reduced_X_valid = X_valid.drop(cols_with_missing, axis="columns")

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']


In [24]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


In [25]:
# Step 3: Imputation

# Part A
# Fill in the lines below: imputation
my_imputer = SimpleImputer() # Your code here
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [26]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18062.894611872147


In [27]:
# Step 4: Generate test predictions

# 1. preprocess training and validation features
# 2. train and evaluate a random forest model
# 3. preprocess the test data before generating predictions

In [28]:
# Step 4, Part A:
# 1. preprocess training and validation features

# Finding all columns within training data that contain null values
cols_with_missing = [
    col for col in X_train.columns
    if X_train[col].isnull().any()
]

# Making copy of training and valid sets so no changes are permanent to orig. dfs
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Making new columns associated with columns that contain null values.
# Column will contain if that row has a null values within that column
for col in cols_with_missing:
  X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
  X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Puting average values of column in place for null values
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus)) #why fit_transform?
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus)) #why transform?

# relabeling columns
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

final_X_train = imputed_X_train_plus
final_X_valid = imputed_X_valid_plus

In [29]:
# Step 4, Part A:
# 2. train and evaluate a random forest model

# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

#Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your apporach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your apporach):
18148.417180365297
